# FRIDA: A Collaborative Robot Painter with a Differentiable, Real2Sim2Real Planning Environment

#### [Peter Schaldenbrand](https://pschaldenbrand.github.io/#about.html), [Jean Oh](https://www.cs.cmu.edu/~./jeanoh/), [Jim McCann](http://www.cs.cmu.edu/~jmccann/)
#### The Robotics Institute, Carnegie Mellon University

There's code for multiple different painting tasks below. Feel free to take a look at the command line arguments to change parameters such as the number of brush strokes used or number of optimization iterations.

# Installation

In [1]:
%load_ext tensorboard

In [ ]:
# !git clone https://github.com/pschaldenbrand/Frida.git

In [3]:
cd /root/content/Frida/src

/root/content/Frida/src


In [4]:
# !git pull origin master

From https://github.com/pschaldenbrand/Frida
 * branch            master     -> FETCH_HEAD
Already up to date.


In [15]:
# !python -m pip install -r ../requirements.txt
!python -m pip install -r ../requirements_python3.txt

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-68wqq5c5
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-68wqq5c5


In [7]:
!pip install opencv-python


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 61.7 MB 833 kB/s eta 0:00:012


In [6]:
# !mkdir painting

In [24]:
# import requests
# from PIL import Image
# from io import BytesIO
# def save_image_from_internet(url, fn):
#     response = requests.get(url)
#     img = Image.open(BytesIO(response.content))
#     img = img.convert('RGB')
#     img.save(fn)

    
import requests
import cv2
import numpy as np

def save_image_from_internet(url, filename):
    response = requests.get(url)
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)  # 读取彩色图像
    cv2.imwrite(filename, image)  # 保存图像

# 示例使用
# save_image_from_internet('https://example.com/image.jpg', 'saved_image.jpg')


# Paint :]

In [8]:
#@title # Conceptually Paint a Reference Image
#@markdown Paint from a source image using feature space loss.
#@markdown Features are extracted from the last convolutional layer of CLIP.

#@markdown Provide the URL to an image you'd like to paint

image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

!wget $image
image = image.split('/')[-1]

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 --use_cache \
        --cache_dir caches/cache_16_20_small2 --n_stroke_models 1 \
        --objective clip_conv_loss \
        --objective_data $image  \
        --objective_weight 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 400 \
        --n_colors 30 \
        --plan_gif_dir 'content/Frida/src/Videos/'

--2023-12-03 22:12:43--  https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg
Resolving cdn.homeandmoney.com (cdn.homeandmoney.com)... 216.137.39.56, 216.137.39.116, 216.137.39.26
Connecting to cdn.homeandmoney.com (cdn.homeandmoney.com)|216.137.39.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 782594 (764K) [image/jpeg]
Saving to: ‘Pittsburgh_FeaturedImg-1.jpg.6’

Pittsburgh_Featured 100%[===================>] 764.25K   785KB/s    in 1.0s    

2023-12-03 22:12:44 (785 KB/s) - ‘Pittsburgh_FeaturedImg-1.jpg.6’ saved [782594/782594]



ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/root/miniconda3/bin/tensorboard", line 5, in <module>
    from tensorboard.main import run_main
  File "/root/miniconda3/lib/python3.8/site-packages/tensorboard/main.py", line 27, in <module>
    from tensorboard import default
  File "/root/miniconda3/lib/python3.8/site-packages/tensorboard/default.py", line 33, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/root/miniconda3/lib/python3.8/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard.data import provider
  File "/root/miniconda3/lib/python3.8/site-packages/tensorboard/data/__init__.py", line 17, in <module>
    from tensorboard.data import experimental  # noqa: F401
  File "/root/miniconda3/lib/python3.8/site-packages/tensorboard/data/experimental/__init__.py", line 17, in <module>
    from tensorboard.data.experimental.experiment_from_dev

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 53.2MiB/s]
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
100%|███████████████████████████████████████| 335M/335M [00:09<00:00, 37.2MiB/s]
Optimizing 784 Strokes:  52%|████████▉        | 210/400 [19:14<18:20,  5.79s/it]/root/miniconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Optimizing 784 Strokes:  55%|█████████▎       | 220/400 [20:09<16:57,  5.65s/it]/root/miniconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'aut

In [ ]:
#@title # Paint from Style and Text Input

#@markdown Paint based off of an example style (give a URL) and a language description of the painting

style_image = "https://media.newyorker.com/photos/5a82035156b75c08a3e5b5b4/4:3/w_2043,h_1532,c_limit/Cunningham-Obama-Kehinde-Wiley.jpg" #@param {type:"string"}
text_description = "A portrait of Samuel L Jackson smiling" #@param {type:"string"}

save_image_from_internet(style_image, 'input0.png')

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 360 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --init_objective l2 --init_objective_data input0.png \
        --init_objective_weight 1.0 \
        --init_optim_iter  40 \
        --objective style text \
        --objective_data input0.png "$text_description" \
        --objective_weight 0.2 1.0 \
        --lr_multiplier 0.4 \
        --n_init 3 \
        --num_strokes 800 --optim_iter 300 \
        --n_colors 30\
        --plan_gif_dir 'content/Frida/src/Videos/'

In [ ]:
#@title # Sketch2Painting with Style and Language Guidance

#@markdown Paint based off of a sketch to guide composition. Also give an example style and text description to help fill in details.

sketch = "https://as2.ftcdn.net/v2/jpg/02/75/70/77/500_F_275707791_2M7x7kOugTyRtsBCrrqpD1fYoLMMEkC7.jpg" #@param {type:"string"}
style_image = "https://uploads1.wikiart.org/images/norman-rockwell/ghostly-gourds.jpg!Large.jpg" #@param {type:"string"}
text_description = "Laura Dern flexin'" #@param {type:"string"}

save_image_from_internet(style_image, 'input0.png')

!wget $sketch
sketch = sketch.split('/')[-1]

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 360 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --objective clip_conv_loss style text \
        --objective_data "$sketch" input0.png "$text_description" \
        --objective_weight 0.7 0.5 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 300 \
        --n_colors 30\
        --plan_gif_dir 'content/Frida/src/Videos/'

In [ ]:
#@title # Image Editing via Language

#@markdown Paint a given image with an edit that you describe

image = "https://vetstreet.brightspotcdn.com/dims4/default/ed822f6/2147483647/thumbnail/645x380/quality/90/?url=https%3A%2F%2Fvetstreet-brightspot.s3.amazonaws.com%2F75%2F371580ca7a11e0ad9e12313817c323%2Ffile%2FBeagle-1-645mk062311.jpg" #@param {type:"string"}
text_edit = "The dog is happy" #@param {type:"string"}

save_image_from_internet(image, 'input0.png')

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --init_objective l2 --init_objective_data input0.png \
        --init_objective_weight 1.0 \
        --init_optim_iter  40 \
        --objective clip_fc_loss text \
        --objective_data input0.png "$text_edit" \
        --objective_weight 0.2 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 300 \
        --n_colors 30\
        --plan_gif_dir 'content/Frida/src/Videos/'

In [26]:
#@title # Paint from an Image with Euclidean Distance
#@markdown Paint from a source image as exactly as possible.
#@markdown The algorithm tries to minimize the $L_2$ distance between the painting and the source iamge.

#@markdown Provide the URL to an image you'd like to paint

image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

save_image_from_internet(image, 'input0.png')

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 --use_cache \
        --cache_dir caches/cache_6_6_4.5cm --n_stroke_models 1 \
        --objective l2 \
        --objective_data input0.png  \
        --objective_weight 1.0 \
        --lr_multiplier 0.4 \
        --n_init 0 \
        --num_strokes 800 --optim_iter 400 \
        --n_colors 30\
        --plan_gif_dir 'content/Frida/src/Videos/'

Reusing TensorBoard on port 6008 (pid 9763), started 0:04:28 ago. (Use '!kill 9763' to kill it.)

Optimizing 784 Strokes:  52%|████████▉        | 210/400 [19:52<18:32,  5.85s/it]/root/miniconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Optimizing 784 Strokes:  55%|█████████▎       | 220/400 [20:49<17:31,  5.84s/it]/root/miniconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Optimizing 784 Strokes:  57%|█████████▊       | 230/400 [21:45<16:39,  5.88s/it]/root/miniconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppr

In [ ]:
#@title # Draw with Ink
#@markdown Paint from a source image using feature space loss.
#@markdown Features are extracted from the last convolutional layer of CLIP.

#@markdown Provide the URL to an image you'd like to paint

image = "https://cdn.homeandmoney.com/wp-content/uploads/2022/05/31113751/Pittsburgh_FeaturedImg-1.jpg" #@param {type:"string"}

!wget $image
image = image.split('/')[-1]

%tensorboard --logdir /content/Frida/src/painting
!python plan.py --simulate --max_height 256 \
        --use_cache --cache_dir caches/cache_10_8_ink --n_stroke_models 1 \
        --objective clip_conv_loss \
        --objective_data $image  \
        --objective_weight 1.0 \
        --lr_multiplier 0.8 \
        --n_init 0 \
        --num_strokes 200 --optim_iter 400 \
        --ink\
        --plan_gif_dir 'content/Frida/src/Videos/'

In [ ]:
 # Generalized Execution
 %tensorboard --logdir /content/Frida/src/painting --reload_interval 10 --reload_multifile True
!python plan.py --simulate --max_height 196 --use_cache --cache_dir cache/ --n_stroke_models 1 \
        --init_objective clip_conv_loss --init_objective_data frida.jpg --init_objective_weight 1.0 \
        --init_optim_iter  40 \
        --objective clip_conv_loss \
        --objective_data frida.jpg \
        --objective_weight 1.0 \
        --lr_multiplier 0.3 \
        --n_init 1\
        --num_strokes 800 --optim_iter 800 \
        --n_colors 12\
        --plan_gif_dir 'content/Frida/src/Videos/'